In [1]:
# Fix versions of the packages we need to ensure JuliaBox compatibility.
import Pkg

Pkg.add([
    Pkg.PackageSpec(name="JuMP", version="0.18.5"),
    Pkg.PackageSpec(name="GLPKMathProgInterface", version="0.4.4")
])

# Import the packages we'll use (JuMP modeller, GLPK solver).
using JuMP
using GLPKMathProgInterface

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


# Part 1: Generating Feasible Crop Schedules

Technical criteria:
* Each crop can only be grown in a subset Ii of the periods.
* Each crop has a specific growing cycle ( ti ).

Set of “ecological-based criteria”:
* Crops of the same botanical family cannot be grown one immediately after
another.
* Use of green manure crops.
* Use of fallow periods (with specific durations).

The variables allow for valid planting times.

The constraints enforce that:
* only one crop is planted in each period,
* crops of the same family do not follow one another,
* one green manuring crop is planted, and
* there is one period of fallow.

The model below defines feasible crop rotation schedules on a single plot.

\begin{alignat*}{2}
& \sum_{i=1}^{N+1} \sum_{r=0}^{t_i-1} x_{i,j-r} \le 1, \quad j = 1 \dots M, \\
& \sum_{i \in F_p} \sum_{r=0}^{t_i} x_{i,j-r} \le 1, \quad p = 1 \dots |F_p|,\, j = 1 \dots M, \\
& \sum_{i\in G} \sum_{j \in I_i} x_{i,j} \le 1, \\
& \sum_{j=1}^{M} x_{N+1,j} = 1, \\
& x_{ij} \in \lbrace 0, 1 \rbrace, \quad i = 1 \dots N + 1,\, j = \in I_i  \\
\end{alignat*}

## Model with JuMP:

In [2]:
# Crop rotation schedule data.
M = 12             # Number of periods in the rotation.
N = 3              # Number of crops.
G = [3]            # Set of green manuring crops.
F = [[1, 2], [3]]  # Set of crop families.
t = [5, 3, 2, 1]   # Production time for each crop.
I = [              # Valid planting times.
    1:4, 1:12,
    [m for m in 1:M if !(m in 4:6)],
    1:12]
n = 4              # Fallow 'crop'.

# Build a model with binary variables x_ij.
# x_ij = 1 => crop i planted in period j.
model = Model(solver=GLPKSolverMIP())
@variable(model, x[i in 1:N+1, j in I[i]], Bin)

x[i,j] ∈ {0,1} ∀ i ∈ {1,2,3,4}, j ∈ {…}

### No overlap of crop plantings

Inspect the expression generated for period 1 below.

In [3]:
j = 1
@constraint(model,
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]-1
            if mod(j - r - 1, M) + 1 in I[i])
        for i in 1:N+1) <= 1)

x[1,1] + x[2,1] + x[2,12] + x[2,11] + x[3,1] + x[3,12] + x[4,1] ≤ 1

The following code generates these constraints for every planting period:

In [4]:
@constraint(model,
    [j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]-1
            if mod(j - r - 1, M) + 1 in I[i])
        for i in 1:N+1) <= 1);

### Crops of the same family may not be planted after one another.

Inspect the expression generated for period 1 and family 1 (containing production crops 1 and 2).

In [5]:
p = 1
j = 1
@constraint(model,
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]
            if mod(j - r - 1, M) + 1 in I[i])
        for i in F[p]) <= 1)

x[1,1] + x[2,1] + x[2,12] + x[2,11] + x[2,10] ≤ 1

This constraint is added to the model for every planting period and every crop family:

In [6]:
@constraint(model,
    [f in F, j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]
            if mod(j - r - 1, M) + 1 in I[i])
        for i in f) <= 1);

### One green manure crop and one period of fallow required.

In [7]:
@constraint(model,
    sum(
        sum(x[i, j] for j in I[i])
        for i in G) == 1)

x[3,1] + x[3,2] + x[3,3] + x[3,7] + x[3,8] + x[3,9] + x[3,10] + x[3,11] + x[3,12] = 1

In [8]:
@constraint(model, sum(x[N + 1, j] for j in 1:M) == 1)

x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] + x[4,6] + x[4,7] + x[4,8] + x[4,9] + x[4,10] + x[4,11] + x[4,12] = 1

## Solve with a simple objective function.

Below assigns a simple objective function to maximise utilisation, which is simply given by

$$\sum_{i=1}^{N} \sum_{j \in I_i} t_i x_{ij}$$

In [9]:
@objective(model, Max,
    sum(
        sum(
            t[i] * x[i, j]
            for j in I[i])
        for i in 1:N))
model

Maximization problem with:
 * 40 linear constraints
 * 37 variables: 37 binary
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

## Solve the resulting model and display the solution

Of the 12 month cycle, 10 months are used for plantings.
Including one forced period of fallow, there is one spare month which couldn't be filled.

In [10]:
println(solve(model))
println("Land Use: ", round(getobjectivevalue(model)))

Optimal
Land Use: 10.0


In [11]:
# Return a string representation of the resulting crop planting times.
function get_schedule(x, M, N)
    schedule = ""
    for j in 1:M
        selected_crop = [
            i for i in 1:N+1
            if j in I[i]
            && (getvalue(x[i, j]) > 0.999)]
        if length(selected_crop) > 0
            if selected_crop[1] == N + 1
                schedule = string(schedule, "|", "F")
            else
                schedule = string(schedule, "|", selected_crop[1])
            end
        else
            schedule = string(schedule, "| ")
        end
    end
    schedule = string(schedule, "|")
end

# Returns the number of plantings of each crop in this schedule.
function get_production(x, M, C)
    [sum(getvalue(x[i, j]) for j in I[i]) for i in 1:N]
end

println("Schedule:   ", get_schedule(x, M, N))
println("Production:   ", get_production(x, M, N))

Schedule:   |3| | |1| | | | |F|2| | |
Production:   [1.0, 1.0, 1.0]


# Part 2: Assigning schedules to plot areas

Generate crop schedules $k \in K$ which produce $c_{ik}$ of crop $i$ per unit area.
By assigning areas to schedules, minimise land use required to satisfy the given demand.

\begin{alignat*}{2}
& \min \,\, \sum_{k \in K} a_{k} \\
& \sum c_{ik} a_{k} \ge d_{i}, \quad i \in C \\
& a_{k} \ge 0, \quad k \in K \\
\end{alignat*}

The subproblem searches for feasible crop schedules with the following objective:

\begin{alignat*}{2}
& z = \min \,\, 1 - \sum_{i \in C} \sum_{j \in I[i]} \lambda_{i} x_{ij} \\
\end{alignat*}

where $\lambda_i$ are the dual variables associated with the demand coverage constraints.

## Define pricing problem as a callable function

This function solves for a crop rotation schedule under the current set of constraints with given dual values.
We'll use this to solve the column generation subproblem which calculates a new column and its reduced costs.

In [12]:
# Construct and solve a crop schedule feasibility model with
# the given objective values applied to a planting of each
# crop type. Returns the resulting objective value and a
# representation of the schedule.
function price_crop_rotation(L)
    # Formulate a new model.
    model = Model(solver=GLPKSolverMIP())
    @variable(model, x[i in 1:N+1, j in I[i]], Bin)
    # One crop at a time.
    @constraint(model,
        [j in 1:M],
        sum(
            sum(
                x[i, mod(j - r - 1, M) + 1]
                for r in 0:t[i]-1
                if mod(j - r - 1, M) + 1 in I[i])
            for i in 1:N+1) <= 1);
    # Crops of same family.
    @constraint(model,
        [f in F, j in 1:M],
        sum(
            sum(
                x[i, mod(j - r - 1, M) + 1]
                for r in 0:t[i]
                if mod(j - r - 1, M) + 1 in I[i])
            for i in f) <= 1);
    # Fallow & green manure.
    @constraint(model,
        sum(
            sum(x[i, j] for j in I[i])
            for i in G) == 1)
    @constraint(model, sum(x[N + 1, j] for j in 1:M) == 1)
    # Parameterised objective coefficients.
    @objective(model, Min,
        1 - sum(
            L[i] * x[i, j]
            for i in 1:N
            for j in I[i]))
    solve(model)
    getobjectivevalue(model), get_production(x, M, N), get_schedule(x, M, N)
end

price_crop_rotation (generic function with 1 method)

Using $t$ as our objective to maximise utilisation, we get our initial schedule which plants one unit of each crop.

$c_k$ is the number of units of demand satisfied per unit area (one planting produces one unit of that crop)

In [13]:
objective, production, schedule = price_crop_rotation(t)
println(objective)
println(schedule)
println("c_k = ", production)

-9.0
|3| | |1| | | | |F|2| | |
c_k = [1.0, 1.0, 1.0]


Value crop 2 more highly - eliminates the planting of crop 1 in favour of a second planting of crop 2.

### Generate a column to start with.

Naively construct some columns, enough to satisfy demand.

In [14]:
# Create some naive columns.
objective, production, schedule = price_crop_rotation([2, 1, 1])
columns = [
    Dict(
        "production" => [1, 0, 1],
        "schedule" => "|1| | | | |F| | |3| | | |"),
    Dict(
        "production" => [0, 1, 1],
        "schedule" => "|2| | |F| | | | |3| | | |")
]

2-element Array{Dict{String,Any},1}:
 Dict("production"=>[1, 0, 1],"schedule"=>"|1| | | | |F| | |3| | | |")
 Dict("production"=>[0, 1, 1],"schedule"=>"|2| | |F| | | | |3| | | |")

In [15]:
# Define demand to be satisfied for each crop.

d = [1, 5, 1]

3-element Array{Int64,1}:
 1
 5
 1

### Construct the initial master problem with the available crop schedules.

There is only one schedule in the initial model, and it produces one unit of each crop.
Each demand constraint therefore enforces at least $d_i$ units of area to satisfy demand for crop $i$.

In [16]:
# Linear master problem.
master_model = Model(solver=GLPKSolverLP())

# Variables assign planting areas.
@variable(master_model, a[i in 1:length(columns)] >= 0)

# Capture the area variables along with the stored column data.
for i in 1:length(columns)
    columns[i]["area"] = a[i]
end

# Uses the production associated with each column to generate coverage constraint.
# Note that this creates an object demand_covered which represents the constraint;
# we can use this to get dual values later.
@constraint(
    master_model, demand_covered[j in 1:length(d)],
    sum(columns[i]["production"][j] * a[i] for i in 1:length(columns)) >= d[j])
@objective(master_model, Min, sum(column["area"] for column in columns))
master_model

Minimization problem with:
 * 3 linear constraints
 * 2 variables
Solver is GLPKMathProgInterface.GLPKInterfaceLP.GLPK

### Obtain the dual values and solve the pricing problem to generate a new column.

\begin{alignat*}{2}
& z = \min \,\, 1 - \sum_{i \in C} \sum_{j \in I[i]} \pi{i} x_{ij} \\
\end{alignat*}

An extra schedule is added which produces more of crop 2, resulting in a new variable added to the master problem which can satisfy the higher demand for that crop.

In [17]:
solve(master_model)
getobjectivevalue(master_model)

6.0

In [18]:
objective, production, schedule = price_crop_rotation(getdual(demand_covered))

(-1.0, [0.0, 2.0, 1.0], "|F|2| | | |2| | | | |3| |")

Objective is less than 0: negative reduced costs indicate an improving column, so add it to the model.

In [19]:
# Create an additional variable.
# This modifies the model so we can reoptimise.
@variable(
    master_model, a_3 >= 0, objective=1.0,
    inconstraints=demand_covered, coefficients=production)
master_model

Minimization problem with:
 * 3 linear constraints
 * 3 variables
Solver is GLPKMathProgInterface.GLPKInterfaceLP.GLPK

In [20]:
# Keep track of the new column data.
new_column = Dict(
    "production" => production,
    "schedule" => schedule,
    "area" => a_3)
push!(columns, new_column)
columns

3-element Array{Dict{String,Any},1}:
 Dict("production"=>[1, 0, 1],"area"=>a[1],"schedule"=>"|1| | | | |F| | |3| | | |")     
 Dict("production"=>[0, 1, 1],"area"=>a[2],"schedule"=>"|2| | |F| | | | |3| | | |")     
 Dict("production"=>[0.0, 2.0, 1.0],"area"=>a_3,"schedule"=>"|F|2| | | |2| | | | |3| |")

Reoptimise the master (total required area reduced from 6 units to 3.5).

In [21]:
solve(master_model)
getobjectivevalue(master_model)

3.5

## Complete Loop

The below handles the full process.

In [22]:
# Create some naive columns.
objective, production, schedule = price_crop_rotation([2, 1, 1])
columns = [
    Dict(
        "production" => [1, 0, 1],
        "schedule" => "|1| | | | |F| | |3| | | |"),
    Dict(
        "production" => [0, 1, 1],
        "schedule" => "|2| | |F| | | | |3| | | |")
]

# Construct the master problem.
master_model = Model(solver=GLPKSolverLP())
@variable(master_model, a[i in 1:length(columns)] >= 0)
for i in 1:length(columns)
    columns[i]["area"] = a[i]
end
@constraint(
    master_model, demand_covered[j in 1:length(d)],
    sum(columns[i]["production"][j] * a[i] for i in 1:length(columns)) >= d[j])
@objective(master_model, Min, sum(column["area"] for column in columns))
master_model

# Run column generation loop to optimality.
while true
    solve(master_model)
    objective, production, schedule = price_crop_rotation(getdual(demand_covered))
    if objective >= 0
        display("DONE")
        break
    end
    @variable(
        master_model, z >= 0, objective=0.0,
        inconstraints=demand_covered, coefficients=production)
    new_column = Dict("production" => production, "schedule" => schedule, "area" => z)
    push!(columns, new_column)
    display("COLUMN ADDED")
    display(new_column)
end

"COLUMN ADDED"

Dict{String,Any} with 3 entries:
  "production" => [0.0, 2.0, 1.0]
  "area"       => z
  "schedule"   => "|F|2| | | |2| | | | |3| |"

"DONE"

### View the resulting schedules

We can see that this assignment over-covers demand for crop 2, so we may be able to add additional columns to reduce area.

In [23]:
for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

|1| | | | |F| | |3| | | |   AREA = 1.0
|2| | |F| | | | |3| | | |   AREA = 0.0
|F|2| | | |2| | | | |3| |   AREA = 2.5
